# LCviz

Load the Kepler 30-minute ("long") cadence light curve of HAT-P-11 from Kepler Quarter 9 (in 2011). HAT-P-11 is a K4V host to a transiting hot Neptune with a 4.8 day period, and a stellar rotation period of 29 days.

In [ ]:
import astropy.units as u
from astropy.time import Time
from lightkurve import search_lightcurve

from lcviz import LCviz

# Download the light curve from MAST:
lc_q9 = search_lightcurve("HAT-P-11", mission="Kepler", cadence="long", quarter=9).download()

# Load the light curve into LCviz:
lcviz = LCviz()
lcviz.load_data(lc_q9)
lcviz.show()

Load and display a second Kepler quarter in the same viewer. We'll pre-process the light curves by flattening them to remove out-of-transit variability due to stellar rotation:

In [ ]:
# Download another Kepler quarter:
lc_q10 = search_lightcurve("HAT-P-11", mission="Kepler", cadence="long", quarter=10).download()

# load two quarters into LCviz:
lcviz = LCviz()
lcviz.load_data(lc_q9, data_label='Q9')
lcviz.load_data(lc_q10, data_label='Q10')
lcviz.show()

Use the `Flatten` plugin to remove low frequency trends from the light curves:

In [ ]:
# use the flatten plugin to flatten each quarter:
flatten = lcviz.plugins['Flatten']

for dataset in flatten.dataset.choices:
    flatten.dataset = dataset
    flatten.window_length = 50
    flatten.flatten();

Show a phase folded light curve, given the ephemeris for the transiting exoplanet HAT-P-11 b:

In [ ]:
# get the origin of the time axis in LCviz:
time_coordinates = lcviz.app.data_collection[0].coords
reference_time = time_coordinates.reference_time

# literature ephemeris for hot Neptune planet HAT-P-11 b:
morris2017_epoch = Time(2454605.89146, format='jd')
morris2017_period = 4.88780258  # days

# phase-fold the transit light curve in an ephemeris viewer:
eph = lcviz.plugins['Ephemeris']
eph.period = morris2017_period
eph.t0 = (
    (morris2017_epoch - reference_time).to_value(time_coordinates.unit) % eph.period
)

# offset the wrapping phase so the transit (at phase 0) displays at center
eph.wrap_at = 0.5